<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clone the repository and cd into it

In [1]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data
!git pull

Cloning into 'evaluating-llm-generated-nlu-data'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 262 (delta 155), reused 170 (delta 79), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 166.73 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/evaluating-llm-generated-nlu-data
Already up to date.


Import the necessary classes and methods

In [2]:
from bin.data_generation.generate_data import DataGenerationModel
from bin.data_generation.construct_prompt import load_prompt
from bin.utils.types import DataSet
from transformers import AutoModelForCausalLM, AutoTokenizer

Initialize Phi-3.5 Model

In [3]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Load the model

In [4]:
phi = DataGenerationModel(model=phi_model, tokenizer=phi_tokenizer)

Generate data with basic prompt

In [ ]:
from bin.data_generation.consts import INTENTS, PROMPT_IDS, NUM_SAMPLES
for prompt_id in PROMPT_IDS:
  dataset = phi.build_dataset_from_intents(prompt_id, INTENTS, NUM_SAMPLES)
  dataset.to_csv(f"{prompt_id}_data.csv", columns=["query", "intent"])
  dataset.to_data_frame()

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
 16%|█▌        | 8/50 [00:30<02:31,  3.60s/it]